In [1]:
import mysql.connector
import getpass, os
import pandas as pd
from mysql.connector import Error

# Conectando a la base de datos

In [2]:
# Definición de parámetros de conexión
connection_params = {
    'host': 'localhost',
    'user': 'cmescobar',
    'database': 'foodb',
    'password': getpass.getpass(prompt='Introduzca la contraseña: ')
}

try:
    connection = mysql.connector.connect(**connection_params)

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()

except Error as e:
    print("Error while connecting to MySQL", e)

Introduzca la contraseña: ········
Connected to MySQL Server version  8.0.26


# Definición de los parámetros de interés

In [3]:
# Definición de los alimentos de interés
food_public_ids = ['FOOD00941', 'FOOD00938', 'FOOD00168', 'FOOD00058', 
                   'FOOD00572', 'FOOD00687', 'FOOD00068', 'FOOD00367', 
                   'FOOD00219', 'FOOD00982', 'FOOD00915', 'FOOD00723', 
                   'FOOD00206', 'FOOD00555', 'FOOD00916', 'FOOD00109', 
                   'FOOD00379', 'FOOD00159', 'FOOD00647', 'FOOD00165', 
                   'FOOD00548', 'FOOD00908', 'FOOD00183', 'FOOD00187', 
                   'FOOD00040']

# Definición de las componentes a revisar (Excel Nico)
compounds_public_id_sheet = ['FDB003010', 'FDB009020', 'FDB000571', 'FDB030727', 
                            'FDB016592', 'FDB013228', 'FDB012292', 'FDB014504', 
                            'FDB023903', 'FDB016251', 'FDB000572', 'FDB012738', 
                            'FDB013567', 'FDB003518', 'FDB003521', 'FDB011814', 
                            'FDB011795', 'FDB003523', 'FDB003191', 'FDB000455', 
                            'FDB000453', 'FDB007057', 'FDB014797', 'FDB002251', 
                            'FDB022939', 'FDB001014', 'FDB000574', 'FDB001223', 
                            'FDB023125', 'FDB001977', 'FDB003729']

# Definición de la lista de nutrientes
nutrients_public_id = []

# Definición de la lista de compuestos final
compounds_public_id = []

# Definición de las componentes de interés en una bebida energética
with open('interest_drink_comps_codes.csv', 'r', encoding='utf8') as file:
    for line in file:
        data = line.strip().split(';')
        
        # Revisando el código de interés
        if 'FDBN' in data[2]:
            nutrients_public_id.append(data[2])
        else:
            compounds_public_id.append(data[2])
            
# Se eliminan posibles duplicados
compounds_public_id = list(set(compounds_public_id))

# Y se limpia la lista de nuevos componentes con los repetidos en la lista original
compounds_public_id_sheet = [i for i in compounds_public_id_sheet 
                             if not i in compounds_public_id]

# Realizando las querys necesarias

## Obteniendo las id de los alimentos

In [5]:
query = f'''SELECT id, name
    FROM foods
    WHERE public_id IN {tuple(food_public_ids)}
    ;
'''

# Ejecutando la query
cursor.execute(query)

# Desplegando la información: Obteniendo la lista de foods_id
foods_id_name = [i for i in cursor.fetchall()]

In [6]:
foods_id_name

[(40, 'Pepper'),
 (58, 'Coffee'),
 (68, 'Turmeric'),
 (109, 'Mentha'),
 (159, 'Rosemary'),
 (165, 'Common sage'),
 (168, 'Winter savory'),
 (183, 'Common thyme'),
 (187, 'Common wheat'),
 (206, 'Ginger'),
 (219, 'Ginseng'),
 (372, 'Ginkgo nuts'),
 (384, 'Horseradish tree'),
 (562, 'Shiitake'),
 (569, 'Maitake'),
 (586, 'Cinnamon'),
 (666, 'Salt'),
 (709, 'Chocolate'),
 (748, 'Hibiscus tea'),
 (940, 'Green tea'),
 (947, 'Guarana'),
 (948, 'Mate'),
 (970, 'Coconut milk'),
 (973, 'Coconut oil'),
 (1014, 'Goji')]

## Obteniendo las id de los nutrientes

In [7]:
query = f'''SELECT id, name, public_id
    FROM nutrients
    WHERE public_id IN {tuple(nutrients_public_id)}
    ;
'''

# Ejecutando la query
cursor.execute(query)

# Desplegando la información: Obteniendo la lista de foods_id
nutrients_id_name = [i for i in cursor.fetchall()]

In [8]:
nutrients_id_name

[(2, 'Proteins', 'FDBN00002'), (3, 'Carbohydrate', 'FDBN00003')]

## Obteniendo las id de las componentes

In [9]:
query = f'''SELECT id, name
    FROM compounds
    WHERE public_id IN {tuple(compounds_public_id)}
    ;
'''

# Ejecutando la query
cursor.execute(query)

# Desplegando la información: Obteniendo la lista de foods_id
compounds_id_name = [i for i in cursor.fetchall()]

In [10]:
print(len(compounds_id_name))
compounds_id_name

21


[(574, 'Pyridoxine'),
 (710, 'Choline'),
 (1014, 'Nicotinic acid'),
 (1223, 'Ascorbic acid'),
 (2100, 'Caffeine'),
 (3514, 'Calcium'),
 (3519, 'Magnesium'),
 (3521, 'Phosphorus'),
 (3522, 'Potassium'),
 (3524, 'Sodium'),
 (3583, 'Copper'),
 (3716, 'Sugars'),
 (3730, 'Zinc'),
 (8425, 'Thiamine'),
 (12163, 'Riboflavine'),
 (13393, 'Water'),
 (13403, 'Selenium'),
 (14507, 'Folic acid'),
 (16258, 'Iron'),
 (23049, 'Cobalamin'),
 (23250, 'Vitamin K1 2,3-epoxide')]

# Y de las nuevas componentes

In [11]:
query = f'''SELECT id, name
    FROM compounds
    WHERE public_id IN {tuple(compounds_public_id_sheet)}
    ;
'''

# Ejecutando la query
cursor.execute(query)

# Desplegando la información: Obteniendo la lista de foods_id
compounds_id_sheet_name = [i for i in cursor.fetchall()]

In [12]:
print(len(compounds_id_sheet_name))
display(compounds_id_sheet_name)
print(list(zip(*compounds_id_sheet_name))[0])

22


[(453, 'Theophylline'),
 (455, 'Theobromine'),
 (571, 'Carnitine'),
 (572, 'L-carnitine'),
 (1977, 'Xanthine'),
 (2251, '(±)-Tryptophan'),
 (3011, 'Dodecanoic acid'),
 (3192, 'Taurine'),
 (7058, 'Terpene'),
 (9021, 'Betaine'),
 (11798, 'Salidroside'),
 (11817, 'Resveratrol'),
 (12295, 'Curcumin'),
 (12741, 'L-Theanine'),
 (13231, 'Coenzyme Q'),
 (13570, 'Limonene'),
 (14800, 'Thymol methyl ether'),
 (16599, 'Chlorophyll'),
 (23102, 'Methylcobalamin'),
 (23288, 'Withanolide'),
 (24066, 'D-Glucurono-6,3-lactone'),
 (30890, 'cinnamaldehyde')]

(453, 455, 571, 572, 1977, 2251, 3011, 3192, 7058, 9021, 11798, 11817, 12295, 12741, 13231, 13570, 14800, 16599, 23102, 23288, 24066, 30890)


# Definición de componentes para cada alimento

In [13]:
sql_queries = [i for i in os.listdir('Queries') if i.endswith('.sql')]
for query_file in sql_queries:
    # Lectura de la query
    with open(f'Queries/Essentials/{query_file}', 'r', encoding='utf8') as file:
        query = file.read()
    
    # Solicitud de la query
    df = pd.read_sql(query, connection)
    
    # Revisando de la lista de campos posibles si es que la tabla 
    # tiene todos los valores de interés
    for comp in compounds_id_name:
        if comp[0] not in tuple(df['source_id']):
            # Rellenar campos vacíos con líneas por defecto
            reg_i = {'source_id': comp[0],
                     'orig_source_name': None,
                     'food_id': df.loc[0][2],
                     'orig_food_common_name': None,
                     'orig_content': None,
                     'orig_min': None,
                     'orig_max': None,
                     'orig_unit': None,
                     'standard_content': None}
            
            # Agregando al dataframe
            df = df.append(reg_i, ignore_index=True)
            
    # Ordenando el dataframe
    df = df.sort_values(by='source_id', axis='index', ignore_index=True)
    
    # Modificando los formatos de las columnas de id
    df = df.astype({'source_id': int, 'food_id': int})
    
    # Finalmente, transformando todos los NaN a None
    df = df.astype(object).replace(np.nan, 'None')    
    
    # Pasando el dataframe a csv
    df.to_csv(f'Summary/Food_Compound_Essentials/{query_file[:-4]}.csv', sep=';', 
              header=True, index=False, encoding='utf8', decimal='.')
    
    print(query_file)
    display(df)

# Definición de nutrientes para cada alimento

In [14]:
sql_queries = [i for i in os.listdir('Queries/Nutrients') if i.endswith('.sql')]
for query_file in sql_queries:
    print(query_file)
    
    # Lectura de la query
    with open(f'Queries/Nutrients/{query_file}', 'r', encoding='utf8') as file:
        query = file.read()
    
    # Solicitud de la query
    df = pd.read_sql(query, connection)
    
    # Si es que está vacío, mejor pasar
    if df.empty:
        continue
    
    # Revisando de la lista de campos posibles si es que la tabla 
    # tiene todos los valores de interés
    for comp in nutrients_id_name:
        if comp[0] not in tuple(df['source_id']):
            # Rellenar campos vacíos con líneas por defecto
            reg_i = {'source_id': comp[0],
                     'orig_source_name': None,
                     'food_id': df.loc[0][2],
                     'orig_food_common_name': None,
                     'orig_content': None,
                     'orig_min': None,
                     'orig_max': None,
                     'orig_unit': None,
                     'standard_content': None}
            
            # Agregando al dataframe
            df = df.append(reg_i, ignore_index=True)
            
    # Ordenando el dataframe
    df = df.sort_values(by='source_id', axis='index', ignore_index=True)
    
    # Modificando los formatos de las columnas de id
    df = df.astype({'source_id': int, 'food_id': int})
    
    # Finalmente, transformando todos los NaN a None
    df = df.astype(object).replace(np.nan, 'None')    
    
    # Pasando el dataframe a csv
    df.to_csv(f'Summary/Food_Compound_Nutrients/{query_file[:-4]}.csv', sep=';', 
              header=True, index=False, encoding='utf8', decimal='.')
    
    display(df)

Chocolate.sql


,source_id,orig_source_name,food_id,orig_food_common_name,orig_content,orig_min,orig_max,orig_unit,standard_content
0,2,"Protein, total",709,"Chocolate, milk",5100.0,None,None,mg/100g,5100.0
1,3,"Carbohydrates, total available",709,"Chocolate, milk",58400.0,None,None,mg/100g,58400.0


Cinnamon.sql


,source_id,orig_source_name,food_id,orig_food_common_name,orig_content,orig_min,orig_max,orig_unit,standard_content
0,2,"Protein, total",586,Cinnamon stick,2866.666667,None,None,mg/100g,2866.666667
1,3,"Carbohydrates, total available",586,Cinnamon stick,54050.000000,None,None,mg/100g,54050.000000


Coffee.sql


...

,source_id,orig_source_name,food_id,orig_food_common_name,orig_content,orig_min,orig_max,orig_unit,standard_content
0,2,Protein,666,"Salt, table",0.0,None,None,mg/100g,0.0
1,3,"Carbohydrate, by difference",666,"Salt, table",0.0,None,None,mg/100g,0.0


Shiitake.sql


,source_id,orig_source_name,food_id,orig_food_common_name,orig_content,orig_min,orig_max,orig_unit,standard_content
0,2,Protein,562,"Mushrooms, shiitake, dried",9580.0,None,None,mg/100g,9580.0
1,3,"Carbohydrate, by difference",562,"Mushrooms, shiitake, dried",75370.0,None,None,mg/100g,75370.0


Turmeric.sql


,source_id,orig_source_name,food_id,orig_food_common_name,orig_content,orig_min,orig_max,orig_unit,standard_content
0,2,Protein,68,"Spices, turmeric, ground",7830.0,None,None,mg/100g,7830.0
1,3,"Carbohydrate, by difference",68,"Spices, turmeric, ground",64930.0,None,None,mg/100g,64930.0


# Definición de componentes adicionales

In [15]:
sql_queries = [i for i in os.listdir('Queries/Plus') if i.endswith('.sql')]
for query_file in sql_queries:
    print(query_file)
    
    # Lectura de la query
    with open(f'Queries/Plus/{query_file}', 'r', encoding='utf8') as file:
        query = file.read()
    
    # Solicitud de la query
    df = pd.read_sql(query, connection)
    
    # Si es que está vacío, mejor pasar
    if df.empty:
        continue
    
    # Revisando de la lista de campos posibles si es que la tabla 
    # tiene todos los valores de interés
    for comp in compounds_id_sheet_name:
        if comp[0] not in tuple(df['source_id']):
            # Rellenar campos vacíos con líneas por defecto
            reg_i = {'source_id': comp[0],
                     'orig_source_name': None,
                     'food_id': df.loc[0][2],
                     'orig_food_common_name': None,
                     'orig_content': None,
                     'orig_min': None,
                     'orig_max': None,
                     'orig_unit': None,
                     'standard_content': None}
            
            # Agregando al dataframe
            df = df.append(reg_i, ignore_index=True)
            
    # Ordenando el dataframe
    df = df.sort_values(by='source_id', axis='index', ignore_index=True)
    
    # Modificando los formatos de las columnas de id
    df = df.astype({'source_id': int, 'food_id': int})
    
    # Finalmente, transformando todos los NaN a None
    df = df.astype(object).replace(np.nan, 'None')    
    
    # Pasando el dataframe a csv
    df.to_csv(f'Summary/Food_Compound_Plus/{query_file[:-4]}.csv', sep=';', 
              header=True, index=False, encoding='utf8', decimal='.')
    
    display(df)

Chocolate.sql


,source_id,orig_source_name,food_id,orig_food_common_name,orig_content,orig_min,orig_max,orig_unit,standard_content
0,453,None,709,None,None,None,None,None,None
1,455,None,709,None,None,None,None,None,None
2,571,None,709,None,None,None,None,None,None
3,572,None,709,None,None,None,None,None,None
4,1977,None,709,None,None,None,None,None,None
5,2251,None,709,"Chocolate, milk",37.11,None,None,mg/100 g,37.11
6,3011,C12:0,709,"Chocolate, milk",187.0,None,None,mg/100g,187.0
7,3192,None,709,None,None,None,None,None,None
8,7058,None,709,None,None,None,None,None,None
9,9021,None,709,None,None,None,None,None,None


Cinnamon.sql


,source_id,orig_source_name,food_id,orig_food_common_name,orig_content,orig_min,orig_max,orig_unit,standard_content
0,453,None,586,None,None,None,None,None,None
1,455,None,586,None,None,None,None,None,None
2,571,None,586,None,None,None,None,None,None
3,572,None,586,None,None,None,None,None,None
4,1977,None,586,None,None,None,None,None,None
5,2251,None,586,Cinnamon stick,0.0,None,None,mg/100 g,0.0
6,3011,C12:0,586,Cinnamon stick,9.0,None,None,mg/100g,9.0
7,3192,None,586,None,None,None,None,None,None
8,7058,None,586,None,None,None,None,None,None
9,9021,None,586,None,None,None,None,None,None


Coffee.sql


...

,source_id,orig_source_name,food_id,orig_food_common_name,orig_content,orig_min,orig_max,orig_unit,standard_content
0,453,None,666,None,None,None,None,None,None
1,455,Theobromine,666,"Salt, table",0.0,None,None,mg/100g,0.0
2,571,None,666,None,None,None,None,None,None
3,572,None,666,None,None,None,None,None,None
4,1977,None,666,None,None,None,None,None,None
5,2251,None,666,None,None,None,None,None,None
6,3011,12:0,666,"Salt, table",0.0,None,None,mg/100g,0.0
7,3192,None,666,None,None,None,None,None,None
8,7058,None,666,None,None,None,None,None,None
9,9021,None,666,None,None,None,None,None,None


Shiitake.sql


,source_id,orig_source_name,food_id,orig_food_common_name,orig_content,orig_min,orig_max,orig_unit,standard_content
0,453,None,562,None,None,None,None,None,None
1,455,Theobromine,562,"Mushrooms, shiitake, dried",0.0,None,None,mg/100g,0.0
2,571,None,562,None,None,None,None,None,None
3,572,None,562,None,None,None,None,None,None
4,1977,None,562,None,None,None,None,None,None
5,2251,None,562,None,None,None,None,None,None
6,3011,12:0,562,"Mushrooms, shiitake, dried",12.0,None,None,mg/100g,12.0
7,3192,None,562,None,None,None,None,None,None
8,7058,None,562,None,None,None,None,None,None
9,9021,None,562,None,None,None,None,None,None


Turmeric.sql


,source_id,orig_source_name,food_id,orig_food_common_name,orig_content,orig_min,orig_max,orig_unit,standard_content
0,453,None,68,None,None,None,None,None,None
1,455,Theobromine,68,"Spices, turmeric, ground",0.0,None,None,mg/100g,0.0
2,571,None,68,None,None,None,None,None,None
3,572,None,68,None,None,None,None,None,None
4,1977,None,68,None,None,None,None,None,None
5,2251,None,68,None,None,None,None,None,None
6,3011,12:0,68,"Spices, turmeric, ground",548.0,None,None,mg/100g,548.0
7,3192,None,68,None,None,None,None,None,None
8,7058,None,68,None,None,None,None,None,None
9,9021,Betaine,68,"Spices, turmeric, ground",9.7,None,None,mg/100g,9.7
